---

## **Introduction to Turbo Codes**

Turbo codes, introduced by Claude Berrou in 1993, were the first practical channel codes to approach the **Shannon capacity limit**. Their breakthrough came from combining two ideas:

* **Parallel concatenation of two convolutional encoders**
* **Iterative decoding with soft information exchange**

They are called “**turbo**” due to the **feedback-like decoding loop**, reminiscent of a turbo engine’s feedback mechanism.


## **Basic Structure**

Turbo coding consists of:

### a. **Two Recursive Systematic Convolutional (RSC) Encoders**

* Connected in parallel
* Both encode the same input, but the second encoder sees an **interleaved** version of the input.

### b. **Interleaver**

* A deterministic or pseudorandom permutation of the input bits
* Introduces randomness to break low-weight input patterns (improves minimum distance)

### c. **Code Rate**

* The basic configuration transmits:

  * The **original input bits** (systematic)
  * One **parity bit** from each encoder
* Code rate:

  $$
  R = \frac{\text{Number of input bits}}{\text{Number of transmitted bits}} = \frac{k}{k + 2k} = \frac{1}{3}
  $$

  (Puncturing can increase it to 1/2, 2/3, etc.)


## **Encoding Process (Step-by-Step)**

1. Input data: $\mathbf{u} = [u_0, u_1, \dots, u_{k-1}]$
2. Encoder 1 takes $\mathbf{u}$ and produces parity bits $\mathbf{p}_1$
3. Interleaver scrambles $\mathbf{u} \rightarrow \mathbf{u}'$
4. Encoder 2 takes $\mathbf{u}'$ and produces parity bits $\mathbf{p}_2$
5. Transmitted bits:

   $$
   \mathbf{x} = \left[ u_0, p_{1,0}, p_{2,0}, u_1, p_{1,1}, p_{2,1}, \dots \right]
   $$


## **Recursive Systematic Convolutional (RSC) Encoder**

Each encoder is usually a **rate-1/2 RSC code**, implemented via:

* Shift registers
* Feedback and feedforward polynomials

For example, a **constraint length 3** encoder might use:

* Generator polynomials: $G_1 = 1 + D + D^2$, $G_2 = 1 + D^2$

These generate both:

* **Systematic bit**: Original input bit
* **Parity bit**: Based on current and previous input bits


## **Turbo Decoder (Iterative Decoding)**

Turbo decoding is performed iteratively using **two Soft-Input Soft-Output (SISO)** decoders:

### Step-by-Step:

1. Each decoder uses the **Log-MAP algorithm** or **Max-Log-MAP** to decode its stream, taking into account **a priori probabilities**.
2. Decoder 1 processes the systematic bits and its parity bits → produces **extrinsic LLRs**.
3. Interleaver permutes these extrinsic values → fed into Decoder 2.
4. Decoder 2 processes interleaved systematic bits and parity → updates extrinsic LLRs.
5. These are **deinterleaved** and passed back to Decoder 1.

Each iteration improves reliability of LLRs. After $N$ iterations, a **hard decision** is made:

$$
\hat{u}_i = \begin{cases}
1 & \text{if } L(u_i) > 0 \\
0 & \text{otherwise}
\end{cases}
$$


## **Mathematical Details of Decoding**

The **Log-MAP algorithm** computes the **log-likelihood ratio (LLR)** for each bit:

$$
L(u_i) = \log \frac{P(u_i = 1 | y)}{P(u_i = 0 | y)}
$$

The LLR is decomposed as:

$$
L(u_i) = L_c(y_i) + L_{a}(u_i) + L_e(u_i)
$$

Where:

* $L_c(y_i)$: channel LLR from received symbols
* $L_a(u_i)$: a priori info from other decoder
* $L_e(u_i)$: extrinsic info passed to other decoder


## **Performance and Properties**

| Property             | Turbo Code                           |
| -------------------- | ------------------------------------ |
| Capacity-Approaching | Yes                                  |
| Good at              | Moderate blocklength (1k–10k)        |
| Drawback             | Error floor at high SNR              |
| Complexity           | Medium-high due to iterations        |
| Used in              | 3G, 4G LTE data channels, deep space |


## **Turbo Code vs. Block Code**

Turbo codes achieve high performance by:

* **Interleaving**, which increases effective free distance
* **Iterative decoding**, improving reliability per iteration
* **Soft information exchange**, unlike traditional block codes


## **Diagram Summary**

```
Input:  u0 u1 u2 u3 ...

  ┌──────────────┐
  │   Encoder 1  │─────→ parity 1
  └──────────────┘
        │
        │ systematic bits → TX
        ▼
  ┌──────────────┐
  │  Interleaver │──→ permuted input
  └──────────────┘
        ▼
  ┌──────────────┐
  │   Encoder 2  │─────→ parity 2
  └──────────────┘
```

At receiver:

* Decoder 1 ⇄ Decoder 2 (via interleaver/deinterleaver)
* Iteratively exchange **LLRs**




--- 

### Custom Turbo Code Implementation 


### 1. **Recursive Systematic Convolutional (RSC) Encoder**

* Constraint length $K = 3$
* Generator polynomials: $G_1 = (1, 1, 1)$, $G_2 = (1, 0, 1)$

    * $G_1$ is the systematic path
    * $G_2$ is the feedback parity path

### 2. **Turbo Encoder**

* Two identical RSC encoders
* One processes original bits
* The other processes interleaved bits

### 3. **Turbo Decoder (Iterative)**

* A basic version of **Soft-In Soft-Out (SISO)** decoding using **Log-MAP** (or approximation)
* Exchange **extrinsic information** between the two decoders
* Use **a priori + channel + extrinsic** LLRs




In [2]:
import numpy as np
import matplotlib.pyplot as plt
# ===========================
# 1. Recursive Systematic Convolutional Encoder (RSC)
# ===========================
class RSCEncoder:
    def __init__(self, g_sys, g_parity):
        """
        g_sys, g_parity: generator polynomials as lists of bits, e.g., [1, 1, 1]
        """
        self.g_sys = np.array(g_sys)
        self.g_parity = np.array(g_parity)
        self.K = len(g_sys)

    def encode(self, bits):
        """
        Encodes a bit array using a rate-1/2 RSC encoder.
        Returns: (systematic_bits, parity_bits)
        """
        state = np.zeros(self.K - 1, dtype=int)
        sys_bits = []
        parity_bits = []

        for bit in bits:
            sys_bits.append(bit)

            # Compute feedback = input XOR feedback terms
            feedback = bit
            for i in range(1, self.K):
                feedback ^= self.g_sys[i] & state[i - 1]

            # Compute parity = feedback XOR parity terms
            parity = feedback * self.g_parity[0]
            for i in range(1, self.K):
                parity ^= self.g_parity[i] & state[i - 1]
            parity_bits.append(parity)

            # Update shift register
            state = np.roll(state, shift=1)
            state[0] = feedback

        return np.array(sys_bits), np.array(parity_bits)


if __name__ == "__main__":
    np.random.seed(0)
    test_bits = np.random.randint(0, 2, 10)
    encoder = RSCEncoder(g_sys=[1, 1, 1], g_parity=[1, 0, 1])
    sys, parity = encoder.encode(test_bits)

    print("Input:        ", test_bits)
    print("Systematic:   ", sys)
    print("Parity:       ", parity)




Input:         [0 1 1 0 1 1 1 1 1 1]
Systematic:    [0 1 1 0 1 1 1 1 1 1]
Parity:        [0 1 0 0 0 1 1 0 1 1]


In [3]:
# ===========================
# 2. Interleaver / Deinterleaver
# ===========================
class Interleaver:
    def __init__(self, pattern):
        """
        pattern: a permutation of indices for interleaving (e.g., np.random.permutation(N))
        """
        self.pattern = np.array(pattern)
        self.inverse_pattern = np.argsort(self.pattern)

    def interleave(self, bits):
        return bits[self.pattern]

    def deinterleave(self, bits):
        return bits[self.inverse_pattern]


if __name__ == "__main__":
    bits = np.array([0, 1, 1, 0, 1, 0])
    pattern = np.random.permutation(len(bits))
    interleaver = Interleaver(pattern)

    interleaved = interleaver.interleave(bits)
    recovered = interleaver.deinterleave(interleaved)

    print("Original Bits:   ", bits)
    print("Interleaved:     ", interleaved)
    print("Deinterleaved:   ", recovered)


Original Bits:    [0 1 1 0 1 0]
Interleaved:      [1 0 1 0 1 0]
Deinterleaved:    [0 1 1 0 1 0]


In [4]:
# ===========================
# 3. Turbo Encoder
# ===========================

class TurboEncoder:
    def __init__(self, g_sys, g_parity, interleaver):
        """
        g_sys, g_parity: generator polynomials
        interleaver: instance of Interleaver
        """
        self.encoder1 = RSCEncoder(g_sys, g_parity)
        self.encoder2 = RSCEncoder(g_sys, g_parity)
        self.interleaver = interleaver

    def encode(self, info_bits):
        """
        Returns:
            sys_bits: Systematic bits (from encoder 1)
            parity1: Parity bits from encoder 1
            parity2: Parity bits from encoder 2 (interleaved input)
            interleaved_bits: Interleaved version of info_bits
        """
        sys_bits, parity1 = self.encoder1.encode(info_bits)
        interleaved_bits = self.interleaver.interleave(info_bits)
        _, parity2 = self.encoder2.encode(interleaved_bits)
        return sys_bits, parity1, parity2, interleaved_bits


if __name__ == "__main__":
    np.random.seed(0)
    info_bits = np.random.randint(0, 2, 10)
    interleaver_pattern = np.random.permutation(len(info_bits))
    interleaver = Interleaver(interleaver_pattern)

    encoder = TurboEncoder(g_sys=[1, 1, 1], g_parity=[1, 0, 1], interleaver=interleaver)
    sys, p1, p2, interleaved = encoder.encode(info_bits)

    print("Info Bits:       ", info_bits)
    print("Systematic Bits: ", sys)
    print("Parity 1:        ", p1)
    print("Parity 2:        ", p2)
    print("Interleaved Bits:", interleaved)


Info Bits:        [0 1 1 0 1 1 1 1 1 1]
Systematic Bits:  [0 1 1 0 1 1 1 1 1 1]
Parity 1:         [0 1 0 0 0 1 1 0 1 1]
Parity 2:         [0 1 0 1 1 1 0 0 0 0]
Interleaved Bits: [0 1 1 1 1 0 1 1 1 1]


In [5]:
# ===========================
# 4. Log-MAP approximation for SISO Decoder (max-log-MAP)
# ===========================
class SISODecoder:
    def __init__(self, g_sys, g_parity, noise_var):
        self.noise_var = noise_var
        self.states = 4  # for constraint length 3
        self.trellis = self._build_trellis(g_sys, g_parity)

    def _build_trellis(self, g_sys, g_parity):
        # Build manually for constraint length = 3
        trellis = {}
        for s in range(4):
            for u in [0, 1]:
                current_state = list(map(int, np.binary_repr(s, width=2)))
                input_bit = u

                # Feedback (RSC)
                fb = input_bit ^ (g_sys[1] & current_state[0]) ^ (g_sys[2] & current_state[1])
                parity = fb ^ (g_parity[1] & current_state[0]) ^ (g_parity[2] & current_state[1])

                next_state = (fb << 1) | current_state[0]
                next_state &= 0b11  # 2-bit state
                trellis[(s, u)] = (next_state, parity)
        return trellis

    def decode(self, y_sys, y_parity, apriori):
        N = len(y_sys)
        alpha = np.full((N + 1, 4), -np.inf)
        beta = np.full((N + 1, 4), -np.inf)
        alpha[0, 0] = 0
        beta[N, :] = 0  # all states equally likely at the end

        # Forward alpha
        for k in range(N):
            for s_prev in range(4):
                for u in [0, 1]:
                    s_next, parity = self.trellis[(s_prev, u)]
                    metric = -0.5 / self.noise_var * (
                        (2*u - 1 - y_sys[k])**2 + (2*parity - 1 - y_parity[k])**2
                    ) + apriori[k]*u
                    alpha[k+1, s_next] = max(alpha[k+1, s_next], alpha[k, s_prev] + metric)

        # Backward beta
        for k in reversed(range(N)):
            for s_next in range(4):
                for s_prev in range(4):
                    for u in [0, 1]:
                        if self.trellis.get((s_prev, u), (None,))[0] == s_next:
                            parity = self.trellis[(s_prev, u)][1]
                            metric = -0.5 / self.noise_var * (
                                (2*u - 1 - y_sys[k])**2 + (2*parity - 1 - y_parity[k])**2
                            ) + apriori[k]*u
                            beta[k, s_prev] = max(beta[k, s_prev], beta[k+1, s_next] + metric)

        # LLR output
        extrinsic = np.zeros(N)
        for k in range(N):
            num = -np.inf
            den = -np.inf
            for s_prev in range(4):
                for u in [0, 1]:
                    s_next, parity = self.trellis[(s_prev, u)]
                    metric = alpha[k, s_prev] + beta[k+1, s_next] + \
                             (-0.5 / self.noise_var * ((2*u - 1 - y_sys[k])**2 + (2*parity - 1 - y_parity[k])**2)) + \
                             apriori[k] * u
                    if u == 1:
                        num = max(num, metric)
                    else:
                        den = max(den, metric)
            extrinsic[k] = num - den

        return extrinsic





In [6]:
# ===========================
# 5. Full Turbo Decoder (Iterative)
# ===========================

class TurboDecoder:
    def __init__(self, g_sys, g_parity, interleaver, noise_var, n_iter=5):
        """
        g_sys, g_parity: generator polynomials
        interleaver: Interleaver instance
        noise_var: noise variance of the AWGN channel
        n_iter: number of decoding iterations
        """
        self.interleaver = interleaver
        self.siso1 = SISODecoder(g_sys, g_parity, noise_var)
        self.siso2 = SISODecoder(g_sys, g_parity, noise_var)
        self.n_iter = n_iter

    def decode(self, rx_sys, rx_p1, rx_p2):
        """
        rx_sys: received noisy systematic (LLR)
        rx_p1: received noisy parity1 (LLR)
        rx_p2: received noisy parity2 (LLR)
        Returns:
            decoded_bits: final hard decision after n_iter iterations
            final_llr: final LLRs from decoder 1
        """
        N = len(rx_sys)
        apriori1 = np.zeros(N)

        for _ in range(self.n_iter):
            # Decoder 1
            llr1 = self.siso1.decode(rx_sys, rx_p1, apriori1)
            extrinsic1 = llr1 - apriori1
            apriori2 = self.interleaver.interleave(extrinsic1)

            # Decoder 2
            inter_rx_sys = self.interleaver.interleave(rx_sys)
            llr2 = self.siso2.decode(inter_rx_sys, rx_p2, apriori2)
            extrinsic2 = llr2 - apriori2
            apriori1 = self.interleaver.deinterleave(extrinsic2)

        final_llr = llr1
        decoded_bits = (final_llr >= 0).astype(int)
        return decoded_bits, final_llr




In [7]:
# ===========================
# Step 6: AWGN simulation with BER measurement and plotting.
# ===========================

class TurboSimulation:
    def __init__(self, g_sys, g_parity, block_len, n_iter=5):
        self.g_sys = g_sys
        self.g_parity = g_parity
        self.block_len = block_len
        self.n_iter = n_iter

    def bpsk_mod(self, bits):
        return 1 - 2 * bits  # 0 → +1, 1 → -1

    def run(self, snr_db, num_blocks=20):
        snr_linear = 10 ** (snr_db / 10)
        noise_var = 1 / (2 * snr_linear)
        sigma = np.sqrt(noise_var)

        total_errors = 0
        total_bits = 0

        for _ in range(num_blocks):
            info_bits = np.random.randint(0, 2, self.block_len)
            interleaver_pattern = np.random.permutation(self.block_len)
            interleaver = Interleaver(interleaver_pattern)

            encoder = TurboEncoder(self.g_sys, self.g_parity, interleaver)
            sys_bits, p1_bits, p2_bits, _ = encoder.encode(info_bits)

            tx_sys = self.bpsk_mod(sys_bits)
            tx_p1 = self.bpsk_mod(p1_bits)
            tx_p2 = self.bpsk_mod(p2_bits)

            rx_sys = tx_sys + sigma * np.random.randn(self.block_len)
            rx_p1  = tx_p1 + sigma * np.random.randn(self.block_len)
            rx_p2  = tx_p2 + sigma * np.random.randn(self.block_len)

            rx_sys_llr = 2 * rx_sys / noise_var
            rx_p1_llr  = 2 * rx_p1  / noise_var
            rx_p2_llr  = 2 * rx_p2  / noise_var

            decoder = TurboDecoder(self.g_sys, self.g_parity, interleaver, noise_var, self.n_iter)
            decoded_bits, _ = decoder.decode(rx_sys_llr, rx_p1_llr, rx_p2_llr)

            total_errors += np.sum(decoded_bits != info_bits)
            total_bits += self.block_len

        ber = total_errors / total_bits
        return ber




In [ ]:
if __name__ == "__main__":
    import matplotlib.pyplot as plt

    sim = TurboSimulation(g_sys=[1,1,1], g_parity=[1,0,1], block_len=5000, n_iter=6)
    snr_range = np.arange(0, 3.5, 0.5)
    ber_values = [sim.run(snr_db,num_blocks = 30) for snr_db in snr_range]

    plt.figure()
    plt.semilogy(snr_range, ber_values, marker='o')
    plt.grid(True, which='both')
    plt.title("Turbo Code BER vs. SNR (Max-Log-MAP, 6 Iterations)")
    plt.xlabel("SNR (dB)")
    plt.ylabel("Bit Error Rate (BER)")
    plt.tight_layout()
    plt.show()

--- 

## **Polar codes** 

Invented by Erdal Arıkan in 2009, are the **first class of error-correcting codes** that are **mathematically proven to achieve the capacity** of binary-input symmetric memoryless channels (B-DMC), such as the Binary Erasure Channel (BEC) and AWGN.

### Core Idea: Channel Polarization

The fundamental idea behind polar codes is called **channel polarization**. Given a communication channel $W$, we construct $N = 2^n$ **synthesized channels** $W_1, W_2, ..., W_N$, such that:

* Some channels become **almost noiseless** ($\text{capacity} \to 1$)
* Others become **completely noisy** ($\text{capacity} \to 0$)

We then:

* Transmit data bits over the **good (reliable)** channels
* Fix the inputs of **bad (unreliable)** channels to known values (called **frozen bits**)

### Polar Encoding

#### a. **Input Vector**

Let:

* $\mathbf{u} = [u_0, u_1, \dots, u_{N-1}]$: message + frozen bits
* $N = 2^n$: block length

#### b. **Polar Transform**

Encoding is done via:

$$
\mathbf{x} = \mathbf{u} \cdot G_N
$$

Where:

* $G_N = B_N \cdot F^{\otimes n}$
* $F = \begin{bmatrix}1 & 0 \\ 1 & 1\end{bmatrix}$
* $F^{\otimes n}$ is the **n-th Kronecker power**
* $B_N$: bit-reversal permutation matrix

The polar transform mixes bits so that some synthesized channels become more reliable than others.


### Frozen vs Information Bits

Once channels are polarized:

* Select **K** most reliable channels → assign **data bits**
* Remaining $N - K$ → assign **frozen bits** (usually zero)

These positions are **precomputed** based on channel type (e.g., AWGN) and SNR using **Bhattacharyya parameters** or **density evolution**.

### Decoding: Successive Cancellation (SC)

SC decoding proceeds **bit by bit**:

$$
\hat{u}_i = \begin{cases}
0 & \text{if } i \text{ is frozen} \\
\arg\max_{u_i} P(u_i | y, \hat{u}_0^{i-1}) & \text{otherwise}
\end{cases}
$$

* Uses **LLR recursion**
* Early decisions affect later ones
* Very fast (complexity $O(N \log N)$)

### Successive Cancellation List (SCL) Decoding

SC decoding suffers from:

* Poor performance at short blocklengths
* High error rate due to early decision errors

**SCL decoding** improves performance by:

* Keeping a **list of the best decoding paths**
* Selecting the final codeword using a **CRC check** (used in 5G NR)

SCL-CRC is the **standard decoding method** in real systems.


### Example: $N = 8$ Polar Code

* Block length $N = 8$
* K = 4 data bits, 4 frozen bits
* Frozen positions: {0, 1, 2, 4}
* Information positions: {3, 5, 6, 7}

place:

* 0s in frozen positions
* message bits in data positions

compute:

$$
\mathbf{x} = \mathbf{u} \cdot G_8
$$

### Polar Code Properties

| Feature      | Description                                      |
| ------------ | ------------------------------------------------ |
| Complexity   | $O(N \log N)$ encoding & decoding                |
| Capacity     | Achieves capacity for B-DMCs                     |
| Performance  | Excellent with SCL-CRC (especially at large $N$) |
| Block Length | Must be power of 2                               |
| Error Floor  | Avoided using SCL decoding                       |

---

### Use in 5G NR

* Polar codes are used for **control channels** in **5G NR**:

  * **PBCH**, **PDCCH**, **PUCCH**, **DCI**
* Data channels use **LDPC codes**

Why Polar for control?

* Low latency
* Short block length efficiency
* Efficient hardware implementation


## Polar vs Turbo vs LDPC

| Feature            | Polar               | Turbo           | LDPC               |
| ------------------ | ------------------- | --------------- | ------------------ |
| Invented           | 2009                | 1993            | 1960s / modernized |
| Decoding           | SC, SCL             | Iterative (MAP) | Iterative (BP)     |
| Capacity-achieving | Yes (B-DMC)         | Near capacity   | Yes (long block)   |
| Used in 5G NR      | Control             | No              | Data (PDSCH)       |
| Performance        | Excellent (SCL-CRC) | Moderate        | Excellent (long)   |



In [9]:
from numpy.polynomial.polynomial import Polynomial

# ===============================
# 1. Polar Transform Matrix
# ===============================
def polar_transform_matrix(N):
    """Generate the Polar transform matrix G_N = B_N * F^{⊗n}"""
    def kronecker_power(F, n):
        result = F
        for _ in range(1, n):
            result = np.kron(result, F)
        return result

    def bit_reversal(n):
        N = 2 ** n
        result = np.arange(N)
        for i in range(N):
            b = '{:0{width}b}'.format(i, width=n)
            result[i] = int(b[::-1], 2)
        return result

    n = int(np.log2(N))
    F = np.array([[1, 0], [1, 1]], dtype=int)
    G = kronecker_power(F, n) % 2
    bit_rev_order = bit_reversal(n)
    G = G[bit_rev_order]
    return G

# ===============================
# 2. Polar Encoder
# ===============================
def polar_encode(u, G_N):
    """Polar encoding: x = u * G_N (mod 2)"""
    return np.mod(u @ G_N, 2)

# ===============================
# 3. Successive Cancellation Decoder (SC)
# ===============================
def sc_decode(llr, frozen_bits):
    N = len(llr)
    u_hat = np.zeros(N, dtype=int)

    def recursive_decode(llr, depth=0):
        n = len(llr)
        if n == 1:
            i = len(u_hat) - len(llr)
            if frozen_bits[i] == 1:
                return np.array([0])
            else:
                return np.array([0 if llr[0] >= 0 else 1])
        else:
            llr_left = np.sign(llr[:n//2]) * np.minimum(np.abs(llr[:n//2]), np.abs(llr[n//2:]))
            llr_right = llr[n//2:] + ((1 - 2 * recursive_decode(llr_left, depth + 1)) * llr[:n//2])
            left_bits = recursive_decode(llr_left, depth + 1)
            right_bits = recursive_decode(llr_right, depth + 1)
            return np.concatenate([(left_bits ^ right_bits), right_bits])

    return recursive_decode(llr)

# ===============================
# 4. Test Case
# ===============================
# Parameters
N = 8  # block length (must be power of 2)
K = 4  # number of data bits
np.random.seed(0)

# Reliability order (for simplicity, manually selected for N=8 and AWGN)
# In practice use Arikan's Bhattacharyya or 5G standards
reliable_indices = [3, 5, 6, 7]  # information bits
frozen_indices = [0, 1, 2, 4]    # frozen bits

# Information bits
info_bits = np.random.randint(0, 2, K)

# Prepare u vector (with frozen bits = 0)
u = np.zeros(N, dtype=int)
u[reliable_indices] = info_bits

# Encoding
G_N = polar_transform_matrix(N)
x = polar_encode(u, G_N)

# BPSK modulation and AWGN
def bpsk(x): return 1 - 2 * x
def add_awgn(x, snr_db):
    snr = 10 ** (snr_db / 10)
    noise_std = np.sqrt(1 / (2 * snr))
    return x + noise_std * np.random.randn(len(x))

snr_db = 3
x_mod = bpsk(x)
y = add_awgn(x_mod, snr_db)
llr = 2 * y * (10 ** (snr_db / 10))  # LLR for BPSK over AWGN

# Frozen bit mask: 1 = frozen, 0 = info
frozen_mask = np.ones(N, dtype=int)
frozen_mask[reliable_indices] = 0

# Decoding
u_hat = sc_decode(llr, frozen_mask)
decoded_info_bits = u_hat[reliable_indices]

# Compute BER
ber = np.sum(decoded_info_bits != info_bits) / K
info_bits, decoded_info_bits, ber


# ===============================
# BER vs SNR for Polar Code with SC Decoding
# ===============================

def simulate_polar_sc(snr_db, N=128, K=64, num_trials=100):
    # Manually select reliability order (simplified)
    reliable_indices = sorted(np.random.choice(np.arange(N), K, replace=False))
    frozen_mask = np.ones(N, dtype=int)
    frozen_mask[reliable_indices] = 0

    G_N = polar_transform_matrix(N)
    total_errors = 0
    total_bits = 0

    for _ in range(num_trials):
        # Generate info bits and prepare u
        info_bits = np.random.randint(0, 2, K)
        u = np.zeros(N, dtype=int)
        u[reliable_indices] = info_bits

        # Encode
        x = polar_encode(u, G_N)
        x_mod = bpsk(x)
        y = add_awgn(x_mod, snr_db)
        llr = 2 * y * (10 ** (snr_db / 10))

        # Decode
        u_hat = sc_decode(llr, frozen_mask)
        decoded_info = u_hat[reliable_indices]

        total_errors += np.sum(decoded_info != info_bits)
        total_bits += K

    return total_errors / total_bits


# ===============================
# Successive Cancellation List (SCL) Decoder with CRC
# ===============================

def crc_check(bits, crc_poly, crc_len):
    """Check if CRC remainder is zero."""
    m = np.append(bits, np.zeros(crc_len, dtype=int))
    poly = Polynomial(m[::-1])
    divisor = Polynomial(crc_poly[::-1])
    remainder = poly % divisor
    return np.all(np.round(remainder.coef[::-1][:crc_len]) == 0)

def generate_crc(bits, crc_poly, crc_len):
    """Append CRC to bits."""
    m = np.append(bits, np.zeros(crc_len, dtype=int))
    poly = Polynomial(m[::-1])
    divisor = Polynomial(crc_poly[::-1])
    remainder = poly % divisor
    crc = np.round(remainder.coef[::-1][:crc_len]).astype(int)
    return np.append(bits, crc)

def scl_decode(llr, frozen_mask, L=8, crc_poly=None, crc_len=0):
    """
    SCL decoder with optional CRC.
    """
    N = len(llr)
    paths = [(np.zeros(N, dtype=int), 0.0)]  # (path, path_metric)

    for i in range(N):
        new_paths = []
        for path, metric in paths:
            if frozen_mask[i]:
                new_path = path.copy()
                new_path[i] = 0
                new_metric = metric + np.log1p(np.exp(-abs(llr[i])))
                new_paths.append((new_path, new_metric))
            else:
                for bit in [0, 1]:
                    new_path = path.copy()
                    new_path[i] = bit
                    if bit == 0:
                        new_metric = metric + np.log1p(np.exp(-abs(llr[i])))
                    else:
                        new_metric = metric + np.log1p(np.exp(abs(llr[i])))
                    new_paths.append((new_path, new_metric))
        paths = sorted(new_paths, key=lambda x: x[1])[:L]

    # CRC check
    if crc_poly is not None:
        for path, _ in paths:
            info_bits = path[frozen_mask == 0]
            if crc_check(info_bits, crc_poly, crc_len):
                return path
        return paths[0][0]  # fallback
    else:
        return paths[0][0]



# ===============================
# Simulation with CRC + SCL
# ==============================
def simulate_polar_scl_crc_fixed_errors(
    snr_db, N=128, K=64, crc_poly=[1,1,0,1], crc_len=3, L=8, target_errors=1000):
    effective_K = K - crc_len
    info_indices = sorted(np.random.choice(np.arange(N), K, replace=False))
    frozen_mask = np.ones(N, dtype=int)
    frozen_mask[info_indices] = 0

    G_N = polar_transform_matrix(N)
    total_errors = 0
    total_bits = 0

    while total_errors < target_errors:
        bits = np.random.randint(0, 2, effective_K)
        bits_with_crc = generate_crc(bits, crc_poly, crc_len)
        u = np.zeros(N, dtype=int)
        u[info_indices] = bits_with_crc

        x = polar_encode(u, G_N)
        x_mod = bpsk(x)
        y = add_awgn(x_mod, snr_db)
        llr = 2 * y * (10 ** (snr_db / 10))

        u_hat = scl_decode(llr, frozen_mask, L=L, crc_poly=crc_poly, crc_len=crc_len)
        decoded_info = u_hat[info_indices][:effective_K]

        total_errors += np.sum(decoded_info != bits)
        total_bits += effective_K

    return total_errors / total_bits




In [ ]:
ber_curve_scl_crc = [
    simulate_polar_scl_crc_fixed_errors(snr, N=128, K=64, crc_len=3, L=8, target_errors=1000)
    for snr in snr_range
]

plt.figure()
plt.semilogy(snr_range, ber_curve_scl_crc, marker='o', label='Polar SCL + CRC (L=8)')
plt.grid(True, which='both')
plt.title("Polar Code BER vs SNR with SCL + CRC (N=128, K=61 + CRC 3)")
plt.xlabel("SNR (dB)")
plt.ylabel("Bit Error Rate (BER)")
plt.legend()
plt.tight_layout()
plt.show()

---

There are several Python libraries that provide implementations of **Turbo**, **LDPC**, and **Polar** codes. Below are the most relevant ones, from research-grade to production-quality:

### **CommPy** (Communications with Python)

* **GitHub**: [https://github.com/veeresht/CommPy](https://github.com/veeresht/CommPy)
* **Status**: Actively used in academia (not full 5G spec compliant)
* **Implements**:

  * Turbo codes (basic RSC, iterative decoding)
  * Convolutional codes (Viterbi)
  * Basic LDPC (with `pyldpc` dependency)
  * ❌ Polar (not included)

---

### **PyLDPC**

* **GitHub**: [https://github.com/flennerhag/pyldpc](https://github.com/flennerhag/pyldpc)
* **Implements**:

  * LDPC: generation, encoding, belief propagation decoding
* **Supports**:

  * Regular/irregular LDPC
  * Custom parity-check matrices

> No Turbo or Polar support. Use for LDPC simulation only.

---

### **Sionna** (NVIDIA) – 5G & Deep Learning Ready

* **GitHub**: [https://github.com/NVIDIA/sionna](https://github.com/NVIDIA/sionna)
* **Platform**: TensorFlow 2.x
* **Implements**:

  * Polar codes (including CRC-aided SC and SCL decoding)
  * LDPC (3GPP 5G NR compliant)
  * Turbo (basic support)
  * 5G NR full PHY chain

> If you're doing **5G** simulations or using **ML + comms**, this is the **most comprehensive and accurate** toolkit available.

---

### **IT++ (via Python bindings)**

* **C++ library with Python wrappers**
* Implements all major FEC codes
* But setup is difficult, and usage is dated

> Not recommended unless you already use it in C++

---

### **OpenFEC**

* LDPC and Turbo codes (3GPP-like)
* High-performance C-based implementation, hard to use directly in Python

---

### PyTorch Implementations

Some GitHub repositories (e.g. TurboAE, PolarNet) offer differentiable implementations:

* **TurboAE**: Turbo autoencoder-based neural decoder
* **PolarNet**: Polar code SC/SCL decoding via deep learning

---

---
---

# LDPC (Low-Density Parity-Check) codes



## 1. Introduction and Motivation

Low-Density Parity-Check (LDPC) codes are **linear block codes** defined by a sparse parity-check matrix. Proposed by Gallager in the 1960s and rediscovered in the 1990s, LDPC codes are known for:

* Near-capacity performance on many channels.
* Efficient iterative decoding via message-passing algorithms.
* Practical use in standards like Wi-Fi, 5G, DVB-S2.


## 2. Code Definition

An LDPC code is a linear block code with parameters $(n, k)$, where:

* **n**: codeword length
* **k**: message length
* **Rate**: $R = \frac{k}{n}$

It is defined by its **parity-check matrix** $H$:

$$
H \cdot \mathbf{c}^{\top} = \mathbf{0} \quad \text{over GF(2)}
$$

where:

* $\mathbf{c}$ is a codeword of length $n$.
* $H$ is an $(n - k) \times n$ sparse binary matrix.

**Low density** means that most entries in $H$ are zero.



## 3. Parity-Check Matrix Properties

A parity-check matrix $H$ is said to be **(d\_v, d\_c)-regular** if:

* Each column has exactly $d_v$ ones.
* Each row has exactly $d_c$ ones.

**Irregular** LDPC codes allow variable column/row weights for improved performance.

**Key property**:

$$
H \cdot \mathbf{c}^{\top} = \mathbf{0} \implies \text{each row of } H \text{ imposes a parity-check constraint}.
$$

Example:

For a row $h_i$:

$$
h_{i1} c_1 + h_{i2} c_2 + \ldots + h_{in} c_n = 0 \pmod{2}.
$$


## 4. Graphical Representation (Tanner Graph)

An LDPC code is naturally represented by a **bipartite graph**:

* **Variable nodes (VNs)**: Correspond to codeword bits.
* **Check nodes (CNs)**: Correspond to parity-check equations.

An edge connects variable node $v_j$ to check node $c_i$ if $H_{i,j} = 1$.

**Advantages of Tanner graph**:

* Makes iterative decoding (belief propagation) intuitive.
* Sparsity ensures tractable complexity.



## 5. Encoding

Although defined by the parity-check matrix $H$, one often needs a **generator matrix** $G$ such that:

$$
\mathbf{c} = \mathbf{u} G
$$

where $\mathbf{u}$ is the message vector.

Obtaining $G$ from $H$:

* Systematic form: $H = [A \mid I]$.
* Solve $A \cdot \mathbf{m}^{\top} = \mathbf{p}^{\top}$ for parity bits.

For large, random LDPC codes, efficient encoding can be non-trivial, requiring special constructions (e.g., approximate lower triangular forms).


## 6. Decoding Algorithms

LDPC codes are typically decoded using **iterative message-passing algorithms** on their Tanner graphs.

**6.1. Sum-Product Algorithm (Belief Propagation)**

* Messages are log-likelihood ratios (LLRs).
* Iteratively update messages between variable and check nodes.

**Outline**:

1. Initialize variable node LLRs with channel observations.
2. For each edge, compute check-to-variable messages using parity-check constraints.
3. For each edge, compute variable-to-check messages using incoming messages from other checks.
4. Update variable node beliefs.
5. Check for parity-check satisfaction.

**Mathematical Form**:

For a BIAWGN channel with noise variance $\sigma^2$:

* Input LLR: $L_i^{(0)} = \frac{2 y_i}{\sigma^2}$.

Check-to-variable message:

$$
L_{c \to v} = 2 \tanh^{-1} \left( \prod_{v' \in N(c) \setminus v} \tanh\left( \frac{L_{v' \to c}}{2} \right) \right)
$$

Variable-to-check message:

$$
L_{v \to c} = L_i^{(0)} + \sum_{c' \in N(v) \setminus c} L_{c' \to v}
$$

**6.2. Min-Sum Approximation**

To reduce complexity:

$$
L_{c \to v} \approx \min_{v' \in N(c) \setminus v} |L_{v' \to c}| \times \prod \text{sign}(L_{v' \to c})
$$



## 7. Performance and Capacity Approaching Property

LDPC codes can approach the Shannon capacity of various channels under belief-propagation decoding.

* For BIAWGN channels, LDPC ensembles can be designed to operate close to capacity.
* Threshold behavior: There exists an SNR threshold below which decoding fails with high probability.



## 8. Design Considerations

**Regular LDPC codes**:

* Simple design.
* Uniform weight distribution.
* Good performance at moderate block lengths.

**Irregular LDPC codes**:

* Nodes with varying degrees.
* Degree distributions optimized to improve convergence.

**Degree distribution polynomials**:

* Variable-node edge perspective: $\lambda(x)$.
* Check-node edge perspective: $\rho(x)$.

Example:

$$
\lambda(x) = \sum_{i} \lambda_i x^{i-1}, \quad \rho(x) = \sum_{i} \rho_i x^{i-1}
$$



## 9. Density Evolution and Threshold Analysis

**Density Evolution (DE)** is a powerful analytical method to evaluate asymptotic performance:

* Tracks the distribution of messages (LLRs) passed in the graph over iterations.
* Finds the **threshold**: the worst channel parameter where decoding still succeeds with vanishing error as block length → ∞.

For BEC (Binary Erasure Channel), DE is exact:

$$
x^{(l+1)} = \epsilon \lambda\left(1 - \rho\left(1 - x^{(l)}\right)\right)
$$

where:

* $x^{(l)}$: erasure probability after $l$ iterations.
* $\epsilon$: channel erasure probability.

Threshold is found as the maximum $\epsilon$ such that $x^{(\infty)} = 0$.



---

## 1. Detailed Derivation of Sum-Product Algorithm (Belief Propagation) on Tanner Graph

### 1.1 Setup

Consider transmission over a **Binary-Input Additive White Gaussian Noise (BIAWGN) channel**.
Transmitted codeword bit $c_i \in \{0,1\}$ is BPSK mapped to $x_i \in \{+1,-1\}$:

$$
x_i = 1 - 2c_i
$$

Received signal:

$$
y_i = x_i + n_i, \quad n_i \sim \mathcal{N}(0, \sigma^2)
$$



### 1.2 Initial LLR Computation

The **channel LLR** for bit $i$ is:

$$
L_i^{(0)} = \log \frac{P(c_i=0|y_i)}{P(c_i=1|y_i)} 
$$

For AWGN channel with BPSK signaling:

$$
L_i^{(0)} = \frac{2y_i}{\sigma^2}
$$

This is the **input message** from the channel to each variable node.


### 1.3 Message Passing in Tanner Graph

The **Tanner graph** is bipartite:

* Variable nodes $v$
* Check nodes $c$

We pass **LLR messages** along edges.

**Notation**:

* $L_{v \to c}$: message from variable node $v$ to check node $c$
* $L_{c \to v}$: message from check node $c$ to variable node $v$


### 1.4 Update Equations

#### (a) Variable Node Update

Variable node $v$ combines all incoming check node messages except from $c$:

$$
L_{v \to c} = L_v^{(0)} + \sum_{c' \in N(v) \setminus c} L_{c' \to v}
$$

* $L_v^{(0)}$ is the **channel LLR**.
* Sum over other connected check nodes.


#### (b) Check Node Update

Check nodes impose parity-check constraints:

$$
\sum_{v \in N(c)} c_v = 0 \mod 2
$$

For binary variables, the **probability** that the parity-check is satisfied given incoming messages can be derived using factor graphs. The check-to-variable message in LLR form is:

$$
L_{c \to v} = 2 \tanh^{-1} \left( \prod_{v' \in N(c) \setminus v} \tanh\left( \frac{L_{v' \to c}}{2} \right) \right)
$$

**Derivation Sketch**:

* The probability of even parity is:

$$
P(\text{even}) = \frac{1}{2} \left(1 + \prod_{v' \neq v} (1 - 2p_{v'}) \right)
$$

* Converting probabilities to LLRs and applying log-sum-exp identities leads to the hyperbolic tangent formula.


### 1.5 Decision

After many iterations, variable node beliefs are:

$$
L_v = L_v^{(0)} + \sum_{c \in N(v)} L_{c \to v}
$$

Decision:

$$
\hat{c}_v = \begin{cases}
0 & \text{if } L_v \geq 0 \\
1 & \text{if } L_v < 0
\end{cases}
$$



## 2. Min-Sum Approximation

To reduce complexity of evaluating $\tanh^{-1}$ and products, the **min-sum approximation** is used.

Original:

$$
L_{c \to v} = 2 \tanh^{-1} \left( \prod_{v' \in N(c) \setminus v} \tanh \frac{L_{v' \to c}}{2} \right)
$$

Approximation:

$$
L_{c \to v} \approx \min_{v' \in N(c) \setminus v} |L_{v' \to c}| \times \prod_{v'} \text{sign}(L_{v' \to c})
$$

Rationale:

* $\tanh^{-1}(x) \approx x$ for small $x$.
* Products of $\tanh$ approximated by minimum in log-domain.



## 3. Practical Example of a Small LDPC Code

### 3.1 Small Parity-Check Matrix

Consider:

$$
H = \begin{bmatrix}
1 & 1 & 0 & 1 & 0 & 0 \\
0 & 1 & 1 & 0 & 1 & 0 \\
1 & 0 & 1 & 0 & 0 & 1
\end{bmatrix}
$$

* $n = 6$ (codeword length)
* $m = 3$ (number of parity-checks)



### 3.2 Tanner Graph

* **Variable nodes**: v1 to v6
* **Check nodes**: c1 to c3

**Edges**:

* c1 connects v1, v2, v4
* c2 connects v2, v3, v5
* c3 connects v1, v3, v6


### 3.3 Parity Constraints

Codewords $\mathbf{c}$ satisfy:

$$
\begin{cases}
c_1 + c_2 + c_4 = 0 \mod 2 \\
c_2 + c_3 + c_5 = 0 \mod 2 \\
c_1 + c_3 + c_6 = 0 \mod 2
\end{cases}
$$



## 4. Encoding Example

LDPC codes are often defined by $H$, but to **encode**, we need $G$.

**Method**: Convert $H$ to systematic form:

Suppose $H = [A \mid I]$ for suitable column permutations.

For example, rearrange:

$$
H' = \begin{bmatrix}
1 & 0 & 1 & 0 & 0 & 1 \\
0 & 1 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 & 0 & 0 
\end{bmatrix}
$$

Systematic form:

$$
H' = [A \mid I_3], \quad A = \text{parity generation matrix}
$$

Encoding:

$$
\mathbf{p} = A \cdot \mathbf{m}^{\top} \pmod{2}
$$

$$
\mathbf{c} = [\mathbf{m} \mid \mathbf{p}]
$$


## 5. Irregular LDPC Code Construction

### 5.1 Degree Distribution

Irregular LDPC codes use variable node and check node **degree distributions**.

**Edge perspective polynomials**:

$$
\lambda(x) = \sum_{i} \lambda_i x^{i-1}
$$

$$
\rho(x) = \sum_{i} \rho_i x^{i-1}
$$

* $\lambda_i$: fraction of edges connected to degree $i$ variable nodes.
* $\rho_i$: fraction of edges connected to degree $i$ check nodes.



### 5.2 Example Degree Distribution

Design for BEC:

$$
\lambda(x) = 0.5x + 0.5x^2
$$

$$
\rho(x) = x^5
$$

Interpretation:

* Half of edges connect to degree 2 variable nodes.
* Half connect to degree 3 variable nodes.
* All check nodes have degree 6.


### 5.3 PEG (Progressive Edge Growth) Construction

One practical construction method:

1. Fix desired degrees for each node from the distribution.
2. Grow the Tanner graph edge by edge:

   * Connect variable node with smallest degree deficit.
   * Choose check node to minimize cycles.

PEG ensures:

* Desired degree distributions.
* Large girth (few short cycles).



## Summary of Derived Equations

**Variable Node Update**:

$$
L_{v \to c} = L_v^{(0)} + \sum_{c' \in N(v) \setminus c} L_{c' \to v}
$$

**Check Node Update (Sum-Product)**:

$$
L_{c \to v} = 2 \tanh^{-1} \left( \prod_{v' \in N(c) \setminus v} \tanh\left(\frac{L_{v' \to c}}{2}\right) \right)
$$

**Check Node Update (Min-Sum)**:

$$
L_{c \to v} \approx \min_{v' \in N(c) \setminus v} |L_{v' \to c}| \times \prod \text{sign}(L_{v' \to c})
$$

**Belief Update**:

$$
L_v = L_v^{(0)} + \sum_{c \in N(v)} L_{c \to v}
$$

**Decision Rule**:

$$
\hat{c}_v = \begin{cases}
0 & \text{if } L_v \geq 0 \\
1 & \text{if } L_v < 0
\end{cases}
$$




---
---

# simulation of LDPC decoding


## Assumptions

* Small (toy) LDPC code to keep it understandable.
* Binary code over GF(2).
* AWGN channel.
* BPSK modulation.
* Sum-Product Algorithm with LLRs.




In [10]:
import numpy as np
from scipy.special import logsumexp
# Define LDPC Parity-Check Matrix
H = np.array([
    [1, 1, 0, 1, 0, 0],
    [0, 1, 1, 0, 1, 0],
    [1, 0, 1, 0, 0, 1]
])
# Generate a Codeword
# For simplicity, use all-zeros codeword (valid under any linear code):
n = H.shape[1]
codeword = np.zeros(n, dtype=int)  # All-zero codeword
# BPSK Modulation and AWGN Channel
def bpsk_mod(bits):
    return 1 - 2 * bits

def awgn_channel(x, snr_db):
    snr_linear = 10**(snr_db/10)
    sigma = np.sqrt(1/(2*snr_linear))
    noise = np.random.randn(len(x)) * sigma
    return x + noise

# Simulation parameters
snr_db = 2
x_tx = bpsk_mod(codeword)
y_rx = awgn_channel(x_tx, snr_db)

# Compute Channel LLRs 
# for BIAWNG LLR = 2y/sigma^2
def compute_llrs(y, snr_db):
    snr_linear = 10**(snr_db/10)
    sigma2 = 1/(2*snr_linear)
    return 2 * y / sigma2

llr_channel = compute_llrs(y_rx, snr_db)

# Sum-Product Decoding on Tanner Graph
# implement message-passing over a fixed number of iterations.
num_checks, num_vars = H.shape
max_iters = 10

# Messages on edges
m_v_to_c = H.astype(float) * llr_channel[None, :]  # Initialize with channel LLRs
m_c_to_v = np.zeros_like(m_v_to_c)

for it in range(max_iters):
    # Check node update
    for c in range(num_checks):
        for v in range(num_vars):
            if H[c, v]:
                neighbors = np.where(H[c, :] == 1)[0]
                neighbors = neighbors[neighbors != v]
                prod = 1.0
                for v_prime in neighbors:
                    val = np.tanh(0.5 * m_v_to_c[c, v_prime])
                    prod *= val
                msg = 2 * np.arctanh(prod + 1e-12)
                m_c_to_v[c, v] = msg

    # Variable node update
    for v in range(num_vars):
        for c in range(num_checks):
            if H[c, v]:
                neighbors = np.where(H[:, v] == 1)[0]
                neighbors = neighbors[neighbors != c]
                msg_sum = llr_channel[v]
                for c_prime in neighbors:
                    msg_sum += m_c_to_v[c_prime, v]
                m_v_to_c[c, v] = msg_sum

# Final Beliefs and Hard Decisions
beliefs = llr_channel.copy()
for v in range(num_vars):
    neighbors = np.where(H[:, v] == 1)[0]
    beliefs[v] += m_c_to_v[neighbors, v].sum()

decoded_bits = (beliefs < 0).astype(int)

# Check Parity
syndrome = H @ decoded_bits % 2
print("Decoded bits:", decoded_bits)
print("Syndrome (should be all zero):", syndrome)






Decoded bits: [0 0 0 0 0 0]
Syndrome (should be all zero): [0 0 0]


BCH, RS, OFEC
FEC for optical 

Designing DSP blocks for optical comm